<a href="https://colab.research.google.com/github/manish021996/Web-Scraping/blob/main/Data_Extraction_and_Text_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing all library
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import os
import glob
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# loading datasets
df = pd.read_excel('/content/drive/MyDrive/Internship/Input.xlsx')

In [4]:
df

,URL_ID,URL
0,123.0,https://insights.blackcoffer.com/rise-of-telem...
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...
4,432.0,https://insights.blackcoffer.com/rise-of-telem...
...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...
111,51844.6,https://insights.blackcoffer.com/what-are-the-...
112,52306.4,https://insights.blackcoffer.com/marketing-dri...


In [5]:
data = []
for index, i in enumerate(df['URL']):
  try:
    response = requests.get(i, headers={"User-Agent": "XY"})
  except:
    print("can't get response of {}".format(index))

  try:
    soup = BeautifulSoup(response.content, 'html.parser')
  except:
    print("can't get page of {}".format(index))

  # Extracting the title of each URL Id
  try:
    title = soup.find('h1').get_text()
  except:
    print("Can't get title of {}".format(index))

  # Extracting the text inside the url
  try:
    # Find the element with the specified class
    article_element = soup.find('div', class_='td-post-content tagdiv-type')

    # Extract the text from all <p> elements within the found element
    paragraphs = article_element.find_all('p')

    # Concatenate the text from all <p> elements
    text = '\n'.join([p.get_text() for p in paragraphs])

  except:
    text = ''
    try:
      for p in soup.find_all('p'):
        text = text +' '+ p.get_text()
    except:
      print("can't get text of {}".format(index))

  data.append([i,title,text])

# Creating a dataframe to store all the outputs
output_df = pd.DataFrame(data, columns = ['URL','Title','Text'])

Can't get title of 24
Can't get title of 37


In [6]:
id = []
for i in df['URL_ID']:
  id.append(i)

output_df.insert(0,'URL_ID',id)

In [7]:
# delecting row with url id 24 and 37 as page of URL_ID 24 and 37 does not exist, as it showing error 404
output_df = output_df.drop(index=[24,37])

In [8]:
file_path = '/content/drive/MyDrive/Internship/StopWords'
read_files = glob.glob(os.path.join(file_path, "*.txt"))

# Create a list to store stopwords
stopwords_list = []

# Read words from each file and append them to the stopwords_list
for file in read_files:
    with open(file, 'r', encoding='latin1') as f:
        words = f.read().lower().split()
        stopwords_list.extend(words)

stopwords_list = list(set(stopwords_list))

In [9]:
'''a function for removing the stopword'''

def remove_stopwords(text):
  # removing the stop words and lowercasing the selected words
  text = [word.lower() for word in text.split() if word.lower() not in stopwords_list]
  # joining the list of words with space separator
  return " ".join(text)

In [10]:
# applying remove stopwords function
output_df['Text'] = output_df['Text'].str.lower().apply(remove_stopwords)

In [11]:
folder_path = "/content/drive/MyDrive/Internship/MasterDictionary"

# List all files in the folder
file_names = os.listdir(folder_path)

# Create two lists to store words from each file
positive_words = []
negative_words = []

# Read words from each file
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)

    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='latin1') as file:
            words = file.read().split()

            # Assign words to the appropriate list based on the file name
            if file_name == 'positive-words.txt':
                positive_words.extend(words)
            elif file_name == 'negative-words.txt':
                negative_words.extend(words)

In [12]:
# counting the positive words present in each url
positive_score = []
for text in output_df['Text']:
  positive_word_in_text = []
  for word in text.split():
    if word in positive_words:
      positive_word_in_text.append(word)
  positive_score.append(len(positive_word_in_text))

In [13]:
# counting the negative words present in each url
negative_score = []
for text in output_df['Text']:
  negative_word_in_text = []
  for word in text.split():
    if word in negative_words:
      negative_word_in_text.append(word)
  negative_score.append(len(negative_word_in_text))

In [14]:
# creating field of postive and neagtive score
output_df['POSITIVE SCORE'] = positive_score
output_df['NEGATIVE SCORE'] = negative_score

In [15]:
# counting the number of sentence present in each url
no_of_sentences = []
for text in output_df['Text']:
  no_of_sentences.append(len(sent_tokenize(text)))

In [16]:
# No of words for each url id
no_of_words = []
for text in output_df['Text']:
  no_of_words.append(len(word_tokenize(text)))

In [17]:
# convert into np.array from maths operations
no_of_sentences = np.array(no_of_sentences)
no_of_words = np.array(no_of_words)

In [18]:
# Polarity Score = (Positive Score – Negative Score)/ ((Positive Score + Negative Score) + 0.000001)
output_df['POLARITY SCORE'] = (output_df['POSITIVE SCORE'] - output_df['NEGATIVE SCORE']) / ((output_df['POSITIVE SCORE'] + output_df['NEGATIVE SCORE']) + 0.000001)

# Subjectivity Score = (Positive Score + Negative Score)/ ((Total Words after cleaning) + 0.000001)
output_df['SUBJECTIVITY SCORE'] = (output_df['POSITIVE SCORE'] + output_df['NEGATIVE SCORE']) / ((no_of_words) + 0.000001)


In [19]:
# crate a list to store all the complex word and sylable count for each url
complex_words = []
sylabble_counts = []

for text in output_df['Text']:
  sylabble_count = 0
  split_txt = text.split()
  complex_count = 0

  for word in split_txt:
    count = 0

    if word.endswith('es') or word.endswith('ed'):
      word = word[:-2]

    for i in range(len(word)):
      if(word[i] == 'a' or word[i] == 'e' or word[i] == 'i' or word[i] == 'o' or word[i] == 'u'):
           count+=1

    sylabble_count+=count

    if(count >2):
        complex_count+=1

  sylabble_counts.append(sylabble_count)
  complex_words.append(complex_count)

In [20]:
# Average Sentence Length = the number of words / the number of sentences
output_df['AVG SENTENCE LENGTH'] = no_of_words/no_of_sentences

# Percentage of Complex words = the number of complex words / the number of words
#Complex words are words in the text that contain more than two syllables.
output_df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/no_of_words

# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
output_df['Fog Index'] = 0.4 * (output_df['AVG SENTENCE LENGTH'] + output_df['PERCENTAGE OF COMPLEX WORDS'])

In [21]:
# getting clean word count by:-
# 1. removing the stop words (using stopwords class of nltk package).
# 2. removing any punctuations like ? ! , . from the word before counting.


sw = stopwords.words('english')
for i in sw:
  stopwords_list.append(i)

# Creating a function to Remove Punctuations

def remove_punct(text):
  # replacing the punctuations with no space, which in effect deletes the punctuation marks
  translator = str.maketrans('', '', string.punctuation)
  # return the text stripped of punctuation marks
  return text.translate(translator)

output_df['Text'] = output_df['Text'].apply(remove_stopwords)
output_df['Text'] = output_df['Text'].apply(remove_punct)

In [22]:
# No of clean words for each url id after removing punctuation and stopword present in nlkt lib
no_of_clean_words = []
for text in output_df['Text']:
  no_of_clean_words.append(len(word_tokenize(text)))

In [23]:
# count of personal noune
count_personal_pronouns = []
personal_pronoun =['i', 'we','my', 'ours','and' 'us']
for text in output_df['Text']:
  count = 0
  for word in text:
    if word in personal_pronoun:
      count+=1
  count_personal_pronouns.append(count)

In [24]:
# count total character in each word
total_characters = []
for text in output_df['Text']:
  characters = 0
  for word in text.split():
    characters+=len(word)
  total_characters.append(characters)


In [25]:
output_df['COMPLEX WORD COUNT'] = complex_words
output_df['WORD COUNT'] = no_of_clean_words
output_df['SYLLABLE PER WORD'] = np.array(sylabble_counts) / no_of_clean_words
output_df['PERSONAL PRONOUNS'] = count_personal_pronouns
output_df['AVG WORD LENGTH'] = np.array(total_characters)/ no_of_clean_words

In [26]:
# delecting columns Title and text as they are of no use
output_df = output_df.drop(columns = ['Title', 'Text'])

In [27]:
# now save the dataframe to the disk
output_df.to_csv('Output_Data.csv')

In [28]:
output_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,Fog Index,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,67,18,0.576471,0.083992,12.810127,0.526680,5.334723,533,837,3.096774,654,7.830346
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,37,11,0.541667,0.147692,13.541667,0.584615,5.650513,190,277,3.129964,206,8.122744
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,20,21,-0.024390,0.049637,11.971014,0.412833,4.953539,341,663,2.568627,408,7.131222
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,32,24,0.142857,0.071979,13.186441,0.448586,5.454011,349,659,2.640364,413,7.267071
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,32,24,0.142857,0.071979,13.186441,0.448586,5.454011,349,659,2.640364,413,7.267071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,5,6,-0.090909,0.038732,13.523810,0.355634,5.551777,101,234,2.337607,119,6.696581
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,21,48,-0.391304,0.065527,22.404255,0.317189,9.088578,334,869,2.378596,453,6.624856
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,65,19,0.547619,0.076572,15.671429,0.388332,6.423904,426,915,2.475410,544,6.881967
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,23,17,0.150000,0.046189,14.677966,0.368360,6.018531,319,681,2.497797,392,6.919236
